In [2]:
import os, sys
!{sys.executable} -m pip install torch

Defaulting to user installation because normal site-packages is not writeable


In [3]:
import os, sys
!{sys.executable} -m pip install torchvision

Defaulting to user installation because normal site-packages is not writeable


In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, Dataset, DataLoader

from torch.utils.data import Subset
from torchvision import datasets, models, transforms, utils
from torch.autograd import Variable
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
cuda_enabled = torch.cuda.is_available()
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#cuda_enabled = False
CUDA_VISIBLE_DEVICES=0,1,2,3
cudnn.benchmark = True

from skimage import io, transform
import matplotlib.pyplot as plt

import os
import pandas as pd
import numpy as np
import glob
from __future__ import print_function, division
import datetime
import sys
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split

from ast import literal_eval

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

plt.ion()   # interactive mode

In [6]:
class BiRNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(BiRNN, self).__init__()
        self.is_training = False
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.num_classes = num_classes
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, bidirectional=True)
        self.fc = nn.Dropout(p=0.5, inplace=False)
        self.fc = nn.Linear(hidden_size*2, num_classes)
        self.linear = nn.Linear(self.hidden_size*2, self.num_classes)
        if cuda_enabled:
            self.lstm = self.lstm.cuda()
            self.fc = self.fc.cuda()
            self.linear = self.linear.cuda()
    
    def forward(self, x):
        # Set initial states
        h0 = Variable(torch.zeros(self.num_layers*2, x.size(0), self.hidden_size)) # 2 for bidirection 
        c0 = Variable(torch.zeros(self.num_layers*2, x.size(0), self.hidden_size))
        if cuda_enabled:
            h0 = h0.cuda()  # 2 for bidirection
            c0 = c0.cuda()
        # Forward propagate LSTM
        out, _ = self.lstm(x, (h0, c0))  # out: tensor of shape (batch_size, seq_length, hidden_size*2)
        
        # Decode hidden state of last time step
        if self.is_training:
            out = self.fc(out[:, -1, :])
        else:
            out = out[:, -1, :]
        # out = F.log_softmax(self.linear(out), dim=1)
        return out

In [7]:
class BodyLandmarksDataset(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, csv_file, transform=None):#root_dir = img_dir
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.landmarks_frame = pd.read_csv(csv_file)
        self.transform = transform

    def __len__(self):
        return len(self.landmarks_frame)#same as image label

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        name = self.landmarks_frame.iloc[idx, 0]
        labels = np.zeros(36)
        for i in range(4):
            labels[i]=name
        labels = torch.Tensor(labels).long()
        landmarks = self.landmarks_frame.iloc[idx, 1:]
        marks = []
        for mark in range(len(landmarks)):
            arr=literal_eval(landmarks[mark])
            arr = np.array(arr)
            marks.append(arr)                         
        marks = torch.Tensor(marks)
        
        sample = {'label': labels, 'marks': marks}
        if self.transform:
            sample = self.transform(sample)
        return sample

In [8]:
bodydataset = BodyLandmarksDataset(csv_file='CSVs/BodyTracking.csv')
bodydataset.__len__(), bodydataset.__getitem__(0)['label'].shape, bodydataset.__getitem__(0)['marks'].shape


(224, torch.Size([36]), torch.Size([181, 36]))

In [9]:
def train_total_dataset(dataset, total_split=0.30): #Training set be 70% of the dataset
    train_idx, total_idx = train_test_split(list(range(len(dataset))), test_size=total_split)
    datasets = {}
    datasets['train'] = Subset(dataset, train_idx)
    datasets['total_split'] = Subset(dataset, total_idx)
    return datasets

# hyperparams for the network
batch_size = 64

print('Starting Body NN dataset -------------------------------------')
body_datasets = train_total_dataset(bodydataset)
body_train = body_datasets['train']
body_test = body_datasets['total_split']
# The original dataset is available in the Subset class
print(len(body_datasets['train'].dataset), body_datasets['train'].dataset)
print(len(body_train), body_train)
print(len(body_test), body_test)

print('Starting Body NN loader -------------------------------------')
body_train_loader = DataLoader(dataset=body_train, batch_size=batch_size, shuffle=True)
body_test_loader = DataLoader(dataset=body_test, batch_size=batch_size, shuffle=False)
print(len(body_train_loader), body_train_loader)
print(len(body_test_loader), body_test_loader)
print('Finish  -------------------------------------')

Starting Body NN dataset -------------------------------------
224 <__main__.BodyLandmarksDataset object at 0x1554c8094130>
156 <torch.utils.data.dataset.Subset object at 0x1555505541f0>
68 <torch.utils.data.dataset.Subset object at 0x155550554ac0>
Starting Body NN loader -------------------------------------
3 <torch.utils.data.dataloader.DataLoader object at 0x1554c1dfd220>
2 <torch.utils.data.dataloader.DataLoader object at 0x1554c1dfde50>
Finish  -------------------------------------


In [10]:
#Init NN
timing = dict()
timing['training'] = datetime.datetime.now()
print(timing['training'])
input_size = 1
sequence_length = 181
hidden_size = 128
num_layers = 2
num_classes = 2  # TODO: Determine this from the data
learning_rate = 0.0001
num_epochs = 300

# The network
body_model = BiRNN(input_size, hidden_size, num_layers, num_classes)
body_model.is_training = True

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(body_model.parameters(), lr=learning_rate)

epoch_loss = 5000000000.

2022-05-08 13:32:11.604728


In [ ]:
epochs = []
losses = []

for epoch in range(num_epochs):
    loss_total = 0.
    iteration_count = 0.
    for i, sample in enumerate(body_train):
        iteration_count += 1.
        labels, marks = sample['label'], sample['marks'] 
        marks = Variable(marks.view(-1, sequence_length, input_size))
        labels = Variable(labels)
        if cuda_enabled:
            marks = marks.cuda()
            labels = labels.cuda()

        # Forward + Backward + Optimize
        optimizer.zero_grad()
        outputs = body_model(marks)

        loss = criterion(outputs, labels)
        loss_total += loss.item()
        loss.backward()
        optimizer.step()

        if (i + 1) % 10 == 0:
            print('Epoch [%d/%d], Step [%d/%d], Loss: %.4f'
                  % (epoch + 1, num_epochs, i + 1, len(body_train) // batch_size, loss.item()))
    current_epoch_loss = loss_total / iteration_count
    print('Epoch %d; loss = %0.4f' % (epoch, current_epoch_loss))
    epochs.append(epoch)
    losses.append(current_epoch_loss)
    epoch_loss = current_epoch_loss
timing['training'] = datetime.datetime.now() - timing['training']
print(timing['training'])


Epoch [1/300], Step [10/2], Loss: 0.6820
Epoch [1/300], Step [20/2], Loss: 0.6708
Epoch [1/300], Step [30/2], Loss: 0.4819
Epoch [1/300], Step [40/2], Loss: 0.4465
Epoch [1/300], Step [50/2], Loss: 0.6215
Epoch [1/300], Step [60/2], Loss: 0.4637
Epoch [1/300], Step [70/2], Loss: 0.2739
Epoch [1/300], Step [80/2], Loss: 0.3399
Epoch [1/300], Step [90/2], Loss: 0.3862
Epoch [1/300], Step [100/2], Loss: 0.0442
Epoch [1/300], Step [110/2], Loss: 0.0385
Epoch [1/300], Step [120/2], Loss: 0.0378
Epoch [1/300], Step [130/2], Loss: 0.0397
Epoch [1/300], Step [140/2], Loss: 0.3981
Epoch [1/300], Step [150/2], Loss: 0.3660
Epoch 0; loss = 0.3609
Epoch [2/300], Step [10/2], Loss: 0.3306
Epoch [2/300], Step [20/2], Loss: 0.3808
Epoch [2/300], Step [30/2], Loss: 0.0833
Epoch [2/300], Step [40/2], Loss: 0.0831
Epoch [2/300], Step [50/2], Loss: 0.3397
Epoch [2/300], Step [60/2], Loss: 0.3179
Epoch [2/300], Step [70/2], Loss: 0.0358
Epoch [2/300], Step [80/2], Loss: 0.3926
Epoch [2/300], Step [90/2], 

In [ ]:
torch.save(body_model.state_dict(), 'ShouldIDrive_body_tracking.pkl')

In [ ]:
plt.plot(epochs, losses)

In [ ]:
body_model.is_training = False
timing['testing'] = datetime.datetime.now()
print('Testing -----------------------------------------------')
correct = 0.0
total = 0.0
body_predicted_list = []
body_label_list = []
for i, sample in enumerate(body_test_loader):#test_loader
    labels, marks = sample['label'], sample['marks'] 
    marks = Variable(marks.view(-1, sequence_length, input_size))
    if cuda_enabled:
        marks = marks.cuda()

    outputs = body_model(marks)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    # print(total)
    for p, l in zip(_, labels):
        body_p = int(np.round(p).item())
        body_l = l[0].item()
        body_predicted_list.append(body_p)
        body_label_list.append(body_l)
        if(body_p == body_l):
            correct += 1.0
    print(correct)
print('Test Accuracy of the model on the {} test images: {} %'.format(total, 100 * correct / total)) 

timing['testing'] = datetime.datetime.now() - timing['testing']
print(timing['testing'])

In [ ]:
print('Confusion Matrix')
print('================')
print(confusion_matrix(body_label_list, body_predicted_list))
print('=============================================')
print('Accuracy = %0.4f' % (accuracy_score(body_label_list, body_predicted_list)))
print('=============================================')